In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, KFold, ShuffleSplit, RepeatedStratifiedKFold, LeaveOneOut, StratifiedKFold, cross_val_score, cross_val_predict, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import accuracy_score,classification_report, f1_score, roc_auc_score
import sklearn.metrics as metrics
from sklearn.ensemble import AdaBoostClassifier
from sklearn.preprocessing import MinMaxScaler, StandardScaler, Normalizer, Binarizer, RobustScaler, OneHotEncoder, LabelEncoder, PowerTransformer, QuantileTransformer
from sklearn.pipeline import Pipeline, make_pipeline
import warnings
warnings.filterwarnings("ignore")
from statistics import mean
from sklearn.metrics import f1_score, accuracy_score, recall_score, precision_score, make_scorer
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import plotly.express as px
import pickle
%run auxiliar.py

# Definir la metrica a utilizar 


In [3]:
scorer = metrics.make_scorer(metrics.f1_score)## Modelado

In [4]:
# Lectura de los datos
data = pd.read_csv("../data/superstore_clean.csv")

In [5]:
df_limpio = data.copy()

In [6]:
list(data.columns.values)

['Education_Graduation',
 'Education_PhD',
 'Education_Master',
 'Education_Basic',
 'Marital_Status_Divorced',
 'Marital_Status_Single',
 'Marital_Status_Married',
 'Income',
 'Kidhome',
 'Teenhome',
 'Recency',
 'MntWines',
 'MntFruits',
 'MntMeatProducts',
 'MntFishProducts',
 'MntSweetProducts',
 'MntGoldProds',
 'NumDealsPurchases',
 'NumWebPurchases',
 'NumCatalogPurchases',
 'NumStorePurchases',
 'NumWebVisitsMonth',
 'Response',
 'Complain',
 'Edad',
 'Dt_Customer_year',
 'Dt_Customer_q',
 'Dt_Customer_m',
 'tiempo_participacion']

In [10]:
seed = 12345

In [7]:
# # Solo con columnas seleccionadas en proceso de seleccion de variables
predictors = df_limpio[['Recency', 'NumStorePurchases', 'NumCatalogPurchases', 'MntWines', 'MntMeatProducts',  'NumWebVisitsMonth','Teenhome', 'Income']]


In [8]:
# X = predictors
X = predictors
y = df_limpio.Response

In [11]:
# Train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30, random_state = seed, stratify=y)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((1565, 8), (671, 8), (1565,), (671,))

In [15]:
%%time
# Definir modelo
cbc = CatBoostClassifier(logging_level = 'Silent', random_state = seed)

# Transformador
scaler = QuantileTransformer()

# Pipeline
pipeline = Pipeline([('Quantile', scaler),
                    ('Modelo', cbc)])

# Crear la grilla
grid = {'Modelo__max_depth': [3,4,5],'Modelo__n_estimators':[100, 200, 300],'Modelo__learning_rate' : [0.01, 0.05, 0.1]}

#cv
cv = StratifiedKFold(n_splits = 5, shuffle = True, random_state = seed)

# Llama a grid search
gscv = GridSearchCV (estimator = pipeline, param_grid = grid, scoring = scorer, cv = cv, verbose = False)

#fit the model
gscv.fit(X_train,y_train)

# Resultados
print(f'Mejor estimador: {gscv.best_estimator_}')
print(f'Mejores parametros: {gscv.best_params_}')
print(f'Mejor score: {gscv.best_score_}')


Mejor estimador: Pipeline(steps=[('Quantile', QuantileTransformer()),
                ('Modelo',
                 <catboost.core.CatBoostClassifier object at 0x7ff520a463d0>)])
Mejores parametros: {'Modelo__learning_rate': 0.1, 'Modelo__max_depth': 4, 'Modelo__n_estimators': 300}
Mejor score: 0.4795711685475063
CPU times: user 28.2 s, sys: 23.7 s, total: 52 s
Wall time: 20.8 s


In [16]:
catboost_t = CatBoostClassifier(
    max_depth = 4,
    n_estimators = 300,
    learning_rate = 0.1,
    scale_pos_weight = 3,
    random_state = seed,
    logging_level = 'Silent' #para que no salga print de iteraciones

)

catboost_t.fit(X_train, y_train,eval_set = (X_test, y_test), early_stopping_rounds = 30,use_best_model = True)

In [17]:
# Resultados en train
res_cat_3 = reportes(catboost_t, X_train, y_train, nombre = "cat train scale")
res_cat_3

,Accuracy,Recall,Roc-Auc,Precision,F1
cat train scale,0.9061,0.7607,0.8462,0.6617,0.7078


In [18]:
# Resultados en train
res_cat_3_t = reportes(catboost_t, X_test, y_test, nombre = "cat train scale")
res_cat_3_t

,Accuracy,Recall,Roc-Auc,Precision,F1
cat train scale,0.8614,0.53,0.7247,0.5354,0.5327


In [19]:
# Guarda el modelo
pickle.dump(catboost_t, open('../web/model_8.pkl','wb'))